### **Fonte:** https://huggingface.co/Salesforce/blip-image-captioning-large
### **Referência:** https://github.com/salesforce/BLIP
### **Objetivo:** utilizar em leitores de tela para gerar descrições quando um site não possuir texto alternativo nas imagens

## Realiza os imports e carrega o modelo

In [1]:
import requests;
import urllib;
import torch;
from lxml import html;
from lxml import etree;
from PIL import Image;
from transformers import BlipProcessor, BlipForConditionalGeneration;

# Utiliza GPU se disponível, senão utiliza CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');

# Carrega o modelo
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large");
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device);

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [3]:
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

inputs = processor(raw_image, return_tensors="pt").to(device)

out = model.generate(**inputs)
print(processor.decode(out[0], skip_special_tokens=True))

woman sitting on the beach with her dog and a cell phone


# Definição das funções para processamento das imagens

In [ ]:
# Função principal do programa. Recebe a URL de um site e apresenta a descrição
# de todas as imagens presentes nele.
def process(website_url, images_limit):
  content = read_html(website_url);
  image_urls = get_images_from_html(content);
  count = 0;

  for url in image_urls:
    prompt = generate_prompt(url);

    if (prompt):
      count += 1;

      print(f"Imagem {count}:");
      display(open_image(url));

      print(f"Descrição: {prompt}");
      print("--//--");

    if (count >= images_limit):
      break;

# Recebe a URL de um site e retorna o html dele em string
def read_html(url):
  url_reader = urllib.request.urlopen(url);
  content = url_reader.read().decode("utf-8");
  url_reader.close();

  return content;

# Recebe um html e retorna uma lista com as imagens presentes nele
def get_images_from_html(string):
  images = list();
  html_str = html.fromstring(string);

  for element in html_str.iter():
    if (element.tag == "img"):
      images.append(element.get("src"));

  return images;

# Recebe a URL de uma imagem e retorna o prompt correspondente a ela
def generate_prompt(img_url):
    try:
      image = open_image(img_url);

      inputs = processor(image, return_tensors="pt").to(device);
      out = model.generate(**inputs);

      return processor.decode(out[0], skip_special_tokens=True);
    except:
      return None;

# Abre uma imagem a partir da URL
def open_image(img_url):
  return Image.open(requests.get(img_url, stream=True).raw).convert('RGB');

# Execução

In [ ]:
process(website_url="https://www.feevale.br/", images_limit=5);

Output hidden; open in https://colab.research.google.com to view.